In [1]:
pip install rdflib

     |████████████████████████████████| 235kB 25.0MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 


In [2]:
%tensorflow_version 1.x

!pip install bert-serving-client
!pip install -U bert-serving-server[http]

!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip
!nohup bert-serving-start -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &


!ls  # you should see uncased_something_.zip


from bert_serving.client import BertClient
bc = BertClient(check_length=False)


TensorFlow 1.x selected.
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 358kB 28.6MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=4751c012b0b0f8e3eb60f76c38c46b47d215c07d648ec605b215ae6b13f0b8c2
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil
--2021-06-21 07:27:43--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.128, 172.253.115.128, 172.253.122.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   200MB/s    in 1.9s    

2021-06-21 07:27:45 (200 MB/s) - ‘uncased_L-12_H-76

In [3]:
# Install required packages.
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.6MB 23.9MB/s 
     |████████████████████████████████| 1.4MB 31.0MB/s 
     |████████████████████████████████| 225kB 25.4MB/s 


In [4]:
import torch
import pandas as pd
import rdflib as rdf
import numpy as np
from torch_geometric.data import (InMemoryDataset, Data)
from collections import Counter

In [5]:
import math

def uniform(size, tensor):
    if tensor is not None:
        bound = 1.0 / math.sqrt(size)
        tensor.data.uniform_(-bound, bound)


def kaiming_uniform(tensor, fan, a):
    if tensor is not None:
        bound = math.sqrt(6 / ((1 + a**2) * fan))
        tensor.data.uniform_(-bound, bound)


def glorot(tensor):
    if tensor is not None:
        stdv = math.sqrt(6.0 / (tensor.size(-2) + tensor.size(-1)))
        tensor.data.uniform_(-stdv, stdv)


def glorot_orthogonal(tensor, scale):
    if tensor is not None:
        torch.nn.init.orthogonal_(tensor.data)
        scale /= ((tensor.size(-2) + tensor.size(-1)) * tensor.var())
        tensor.data *= scale.sqrt()


def zeros(tensor):
    if tensor is not None:
        tensor.data.fill_(0)


def ones(tensor):
    if tensor is not None:
        tensor.data.fill_(1)


def normal(tensor, mean, std):
    if tensor is not None:
        tensor.data.normal_(mean, std)


def reset(nn):
    def _reset(item):
        if hasattr(item, 'reset_parameters'):
            item.reset_parameters()

    if nn is not None:
        if hasattr(nn, 'children') and len(list(nn.children())) > 0:
            for item in nn.children():
                _reset(item)
        else:
            _reset(nn)


In [6]:
from copy import copy
from typing import Optional

import torch
from torch import Tensor
from torch_sparse import SparseTensor


@torch.jit._overload
def maybe_num_nodes(edge_index, num_nodes=None):
    # type: (Tensor, Optional[int]) -> int
    pass


@torch.jit._overload
def maybe_num_nodes(edge_index, num_nodes=None):
    # type: (SparseTensor, Optional[int]) -> int
    pass


def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1
    else:
        return max(edge_index.size(0), edge_index.size(1))


def maybe_num_nodes_dict(edge_index_dict, num_nodes_dict=None):
    num_nodes_dict = {} if num_nodes_dict is None else copy(num_nodes_dict)

    found_types = list(num_nodes_dict.keys())

    for keys, edge_index in edge_index_dict.items():

        key = keys[0]
        if key not in found_types:
            N = int(edge_index[0].max() + 1)
            num_nodes_dict[key] = max(N, num_nodes_dict.get(key, N))

        key = keys[-1]
        if key not in found_types:
            N = int(edge_index[1].max() + 1)
            num_nodes_dict[key] = max(N, num_nodes_dict.get(key, N))

    return num_nodes_dict


In [7]:
import random
from torch_geometric.utils import degree, to_undirected


def sample(high: int, size: int, device=None):
    size = min(high, size)
    return torch.tensor(random.sample(range(high), size), device=device)


def negative_sampling(edge_index, num_nodes=None, num_neg_samples=None,
                      method="sparse", force_undirected=False):
    r"""Samples random negative edges of a graph given by :attr:`edge_index`.

    Args:
        edge_index (LongTensor): The edge indices.
        num_nodes (int, optional): The number of nodes, *i.e.*
            :obj:`max_val + 1` of :attr:`edge_index`. (default: :obj:`None`)
        num_neg_samples (int, optional): The (approximate) number of negative
            samples to return. If set to :obj:`None`, will try to return a
            negative edge for every positive edge. (default: :obj:`None`)
        method (string, optional): The method to use for negative sampling,
            *i.e.*, :obj:`"sparse"` or :obj:`"dense"`.
            This is a memory/runtime trade-off.
            :obj:`"sparse"` will work on any graph of any size, while
            :obj:`"dense"` can perform faster true-negative checks.
            (default: :obj:`"sparse"`)
        force_undirected (bool, optional): If set to :obj:`True`, sampled
            negative edges will be undirected. (default: :obj:`False`)

    :rtype: LongTensor
    """

    num_nodes = maybe_num_nodes(edge_index, num_nodes)
    num_neg_samples = num_neg_samples or edge_index.size(1)

    # Handle '|V|^2 - |E| < |E|'.
    size = num_nodes * num_nodes
    num_neg_samples = min(num_neg_samples, size - edge_index.size(1))

    row, col = edge_index

    if force_undirected:
        num_neg_samples = num_neg_samples // 2

        # Upper triangle indices: N + ... + 1 = N (N + 1) / 2
        size = (num_nodes * (num_nodes + 1)) // 2

        # Remove edges in the lower triangle matrix.
        mask = row <= col
        row, col = row[mask], col[mask]

        # idx = N * i + j - i * (i+1) / 2
        idx = row * num_nodes + col - row * (row + 1) // 2
    else:
        idx = row * num_nodes + col

    # Percentage of edges to oversample so that we are save to only sample once
    # (in most cases).
    alpha = abs(1 / (1 - 1.1 * (edge_index.size(1) / size)))

    if method == 'dense':
        mask = edge_index.new_ones(size, dtype=torch.bool)
        mask[idx] = False
        mask = mask.view(-1)

        perm = sample(size, int(alpha * num_neg_samples),
                      device=edge_index.device)
        perm = perm[mask[perm]][:num_neg_samples]

    else:
        perm = sample(size, int(alpha * num_neg_samples))
        mask = torch.from_numpy(np.isin(perm, idx.to('cpu'))).to(torch.bool)
        perm = perm[~mask][:num_neg_samples].to(edge_index.device)

    if force_undirected:
        # (-sqrt((2 * N + 1)^2 - 8 * perm) + 2 * N + 1) / 2
        row = torch.floor((-torch.sqrt((2. * num_nodes + 1.)**2 - 8. * perm) +
                           2 * num_nodes + 1) / 2)
        col = perm - row * (2 * num_nodes - row - 1) // 2
        neg_edge_index = torch.stack([row, col], dim=0).long()
        neg_edge_index = to_undirected(neg_edge_index)
    else:
        row = perm // num_nodes
        col = perm % num_nodes
        neg_edge_index = torch.stack([row, col], dim=0).long()

    return neg_edge_index


def structured_negative_sampling(edge_index, num_nodes=None):
    r"""Samples a negative edge :obj:`(i,k)` for every positive edge
    :obj:`(i,j)` in the graph given by :attr:`edge_index`, and returns it as a
    tuple of the form :obj:`(i,j,k)`.

    Args:
        edge_index (LongTensor): The edge indices.
        num_nodes (int, optional): The number of nodes, *i.e.*
            :obj:`max_val + 1` of :attr:`edge_index`. (default: :obj:`None`)

    :rtype: (LongTensor, LongTensor, LongTensor)
    """
    num_nodes = maybe_num_nodes(edge_index, num_nodes)

    i, j = edge_index.to('cpu')
    idx_1 = i * num_nodes + j

    k = torch.randint(num_nodes, (i.size(0), ), dtype=torch.long)
    idx_2 = i * num_nodes + k

    mask = torch.from_numpy(np.isin(idx_2, idx_1)).to(torch.bool)
    rest = mask.nonzero(as_tuple=False).view(-1)
    while rest.numel() > 0:  # pragma: no cover
        tmp = torch.randint(num_nodes, (rest.numel(), ), dtype=torch.long)
        idx_2 = i[rest] * num_nodes + tmp
        mask = torch.from_numpy(np.isin(idx_2, idx_1)).to(torch.bool)
        k[rest] = tmp
        rest = rest[mask.nonzero(as_tuple=False).view(-1)]

    return edge_index[0], edge_index[1], k.to(edge_index.device)


def batched_negative_sampling(edge_index, batch, num_neg_samples=None,
                              method="sparse", force_undirected=False):
    r"""Samples random negative edges of multiple graphs given by
    :attr:`edge_index` and :attr:`batch`.

    Args:
        edge_index (LongTensor): The edge indices.
        batch (LongTensor): Batch vector
            :math:`\mathbf{b} \in {\{ 0, \ldots, B-1\}}^N`, which assigns each
            node to a specific example.
        num_neg_samples (int, optional): The number of negative samples to
            return. If set to :obj:`None`, will try to return a negative edge
            for every positive edge. (default: :obj:`None`)
        method (string, optional): The method to use for negative sampling,
            *i.e.*, :obj:`"sparse"` or :obj:`"dense"`.
            This is a memory/runtime trade-off.
            :obj:`"sparse"` will work on any graph of any size, while
            :obj:`"dense"` can perform faster true-negative checks.
            (default: :obj:`"sparse"`)
        force_undirected (bool, optional): If set to :obj:`True`, sampled
            negative edges will be undirected. (default: :obj:`False`)

    :rtype: LongTensor
    """
    split = degree(batch[edge_index[0]], dtype=torch.long).tolist()
    edge_indices = torch.split(edge_index, split, dim=1)
    num_nodes = degree(batch, dtype=torch.long)
    cum_nodes = torch.cat([batch.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]])

    neg_edge_indices = []
    for edge_index, N, C in zip(edge_indices, num_nodes.tolist(),
                                cum_nodes.tolist()):
        neg_edge_index = negative_sampling(edge_index - C, N, num_neg_samples,
                                           method, force_undirected) + C
        neg_edge_indices.append(neg_edge_index)

    return torch.cat(neg_edge_indices, dim=1)


In [8]:
from typing import Optional, Union, Tuple
from torch_geometric.typing import OptTensor, Adj

from torch import Tensor
import torch.nn.functional as F
from torch.nn import Parameter as Param
from torch.nn import Parameter
from torch_scatter import scatter
from torch_sparse import SparseTensor, matmul, masked_select_nnz
from torch_geometric.nn.conv import MessagePassing
import logging 
#logging.basicConfig(filename='RGCNConv.log', filemode='a', format='%(asctime)s %(message)s')

@torch.jit._overload
def masked_edge_index(edge_index, edge_mask):
    # type: (Tensor, Tensor) -> Tensor
    pass


@torch.jit._overload
def masked_edge_index(edge_index, edge_mask):
    # type: (SparseTensor, Tensor) -> SparseTensor
    pass


def masked_edge_index(edge_index, edge_mask):
#    logging.warning(f'edge_index_shape is = {edge_index.shape}, edge_mask_shape is = {edge_mask.shape} and edge_index[:, edge_mask] is = {edge_index[:, edge_mask]}')
    if isinstance(edge_index, Tensor):
        #print("If edge_index is a Tensor " + str(edge_index.type))
        return edge_index[:, edge_mask]
    else:
        #print("Otherwise")
        return masked_select_nnz(edge_index, edge_mask, layout='coo')


class RGCNConv(MessagePassing):
    r"""The relational graph convolutional operator from the `"Modeling
    Relational Data with Graph Convolutional Networks"
    <https://arxiv.org/abs/1703.06103>`_ paper

    .. math::
        \mathbf{x}^{\prime}_i = \mathbf{\Theta}_{\textrm{root}} \cdot
        \mathbf{x}_i + \sum_{r \in \mathcal{R}} \sum_{j \in \mathcal{N}_r(i)}
        \frac{1}{|\mathcal{N}_r(i)|} \mathbf{\Theta}_r \cdot \mathbf{x}_j,

    where :math:`\mathcal{R}` denotes the set of relations, *i.e.* edge types.
    Edge type needs to be a one-dimensional :obj:`torch.long` tensor which
    stores a relation identifier
    :math:`\in \{ 0, \ldots, |\mathcal{R}| - 1\}` for each edge.

    .. note::
        This implementation is as memory-efficient as possible by iterating
        over each individual relation type.
        Therefore, it may result in low GPU utilization in case the graph has a
        large number of relations.
        As an alternative approach, :class:`FastRGCNConv` does not iterate over
        each individual type, but may consume a large amount of memory to
        compensate.
        We advise to check out both implementations to see which one fits your
        needs.

    Args:
        in_channels (int or tuple): Size of each input sample. A tuple
            corresponds to the sizes of source and target dimensionalities.
            In case no input features are given, this argument should
            correspond to the number of nodes in your graph.
        out_channels (int): Size of each output sample.
        num_relations (int): Number of relations.
        num_bases (int, optional): If set to not :obj:`None`, this layer will
            use the basis-decomposition regularization scheme where
            :obj:`num_bases` denotes the number of bases to use.
            (default: :obj:`None`)
        num_blocks (int, optional): If set to not :obj:`None`, this layer will
            use the block-diagonal-decomposition regularization scheme where
            :obj:`num_blocks` denotes the number of blocks to use.
            (default: :obj:`None`)
        aggr (string, optional): The aggregation scheme to use
            (:obj:`"add"`, :obj:`"mean"`, :obj:`"max"`).
            (default: :obj:`"mean"`)
        root_weight (bool, optional): If set to :obj:`False`, the layer will
            not add transformed root node features to the output.
            (default: :obj:`True`)
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.MessagePassing`.
    """
    def __init__(self, in_channels: Union[int, Tuple[int, int]],
                 out_channels: int,
                 num_relations: int,
                 num_bases: Optional[int] = None,
                 num_blocks: Optional[int] = None,
                 aggr: str = 'mean',
                 root_weight: bool = True,
                 bias: bool = True, **kwargs):  # yapf: disable

        super(RGCNConv, self).__init__(aggr=aggr, node_dim=0, **kwargs)

        if num_bases is not None and num_blocks is not None:
            raise ValueError('Can not apply both basis-decomposition and '
                             'block-diagonal-decomposition at the same time.')

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_relations = num_relations
        self.num_bases = num_bases
        self.num_blocks = num_blocks

        if isinstance(in_channels, int):
            in_channels = (in_channels, in_channels)
        self.in_channels_l = in_channels[0]

        if num_bases is not None:
            self.weight = Parameter(
                torch.Tensor(num_bases, in_channels[0], out_channels))
            self.comp = Parameter(torch.Tensor(num_relations, num_bases))

        elif num_blocks is not None:
            assert (in_channels[0] % num_blocks == 0
                    and out_channels % num_blocks == 0)
            self.weight = Parameter(
                torch.Tensor(num_relations, num_blocks,
                             in_channels[0] // num_blocks,
                             out_channels // num_blocks))
            self.register_parameter('comp', None)

        else:
            self.weight = Parameter(
                torch.Tensor(num_relations, in_channels[0], out_channels))
            self.register_parameter('comp', None)

        if root_weight:
            self.root = Param(torch.Tensor(in_channels[1], out_channels))
        else:
            self.register_parameter('root', None)

        if bias:
            self.bias = Param(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        glorot(self.comp)
        glorot(self.root)
        zeros(self.bias)

    def forward(self, x: Union[OptTensor, Tuple[OptTensor, Tensor]],
                edge_index: Adj, edge_type: OptTensor = None):
        r"""
        Args:
            x: The input node features. Can be either a :obj:`[num_nodes,
                in_channels]` node feature matrix, or an optional
                one-dimensional node index tensor (in which case input features
                are treated as trainable node embeddings).
                Furthermore, :obj:`x` can be of type :obj:`tuple` denoting
                source and destination node features.
            edge_type: The one-dimensional relation type/index for each edge in
                :obj:`edge_index`.
                Should be only :obj:`None` in case :obj:`edge_index` is of type
                :class:`torch_sparse.tensor.SparseTensor`.
                (default: :obj:`None`)
        """
        logging.warning(f'edge_index_in_forward_start is = {edge_index.shape}')
        # Convert input features to a pair of node features or node indices.
        #print("x has shape: " + str(x.shape))       
        x_l: OptTensor = None
        if isinstance(x, tuple):
            x_l = x[0]
        else:
            x_l = x
        if x_l is None:
            x_l = torch.arange(self.in_channels_l, device=self.weight.device)

        x_r: Tensor = x_l
        if isinstance(x, tuple):
            x_r = x[1]
        #print(x_l)
        size = (x_l.size(0), x_r.size(0))

        if isinstance(edge_index, SparseTensor):
            edge_type = edge_index.storage.value()  
        assert edge_type is not None

        # propagate_type: (x: Tensor)
        out = torch.zeros(x_r.size(0), self.out_channels, device=x_r.device)

        weight = self.weight
        if self.num_bases is not None:  # Basis-decomposition =================
            weight = (self.comp @ weight.view(self.num_bases, -1)).view(
                self.num_relations, self.in_channels_l, self.out_channels)

        if self.num_blocks is not None:  # Block-diagonal-decomposition =====

            if x_l.dtype == torch.long and self.num_blocks is not None:
                raise ValueError('Block-diagonal decomposition not supported '
                                 'for non-continuous input features.')

            for i in range(self.num_relations):
                tmp = masked_edge_index(edge_index, edge_type == i)
                h = self.propagate(tmp, x=x_l, size=size)
                h = h.view(-1, weight.size(1), weight.size(2))
                h = torch.einsum('abc,bcd->abd', h, weight[i])
                out += h.contiguous().view(-1, self.out_channels)

        else: # No regularization/Basis-decomposition ========================
            for i in range(self.num_relations):
                #print(masked_edge_index(edge_index, edge_type == i))
                #logging.warning(f'edge_index is = {edge_index}, edge_type_dim = {(edge_type.shape)} ,edge_type_i is = {edge_type == i} and edge_type_i_dim = {(edge_type == i).shape})') 
                tmp = masked_edge_index(edge_index, edge_type == i) 
                #logging.warning(f'tmp is = {tmp},size is = {size}') 
                if x_l.dtype == torch.long:

                   out += self.propagate(tmp, x=weight[i, x_l], size=size)
                  
                    #out += self.propagate(edge_index, x=weight[i, x_l], size=size)
                else:
                   
                    #print("Size is:" + str(size))
                    #logging.warning(f'tmp is = {tmp.shape} and x_l is = {x_l.shape} and size is = {size}') 
                    h = self.propagate(tmp, x=x_l, size=size)
                    #h = self.propagate(edge_index, x=x_l, size=size)
                    out = out + (h @ weight[i])

        root = self.root
        if root is not None:
            out += root[x_r] if x_r.dtype == torch.long else x_r @ root

        if self.bias is not None:
            out += self.bias

        return out

    def message(self, x_j: Tensor) -> Tensor:
        return x_j

    def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
        adj_t = adj_t.set_value(None, layout=None)
        return matmul(adj_t, x, reduce=self.aggr)

    def __repr__(self):
        return '{}({}, {}, num_relations={})'.format(self.__class__.__name__,
                                                     self.in_channels,
                                                     self.out_channels,
                                                     self.num_relations)


class FastRGCNConv(RGCNConv):
    r"""See :class:`RGCNConv`."""
    def forward(self, x: Union[OptTensor, Tuple[OptTensor, Tensor]],
                edge_index: Adj, edge_type: OptTensor = None):
        """"""
        self.fuse = False
        assert self.aggr in ['add', 'sum', 'mean']

        # Convert input features to a pair of node features or node indices.
        x_l: OptTensor = None
        if isinstance(x, tuple):
            x_l = x[0]
        else:
            x_l = x
        if x_l is None:
            x_l = torch.arange(self.in_channels_l, device=self.weight.device)

        x_r: Tensor = x_l
        if isinstance(x, tuple):
            x_r = x[1]

        size = (x_l.size(0), x_r.size(0))

        # propagate_type: (x: Tensor, edge_type: OptTensor)
        out = self.propagate(edge_index, x=x_l, edge_type=edge_type, size=size)

        root = self.root
        if root is not None:
            out += root[x_r] if x_r.dtype == torch.long else x_r @ root

        if self.bias is not None:
            out += self.bias

        return out

    def message(self, x_j: Tensor, edge_type: Tensor, index: Tensor) -> Tensor:
        weight = self.weight
        if self.num_bases is not None:  # Basis-decomposition =================
            weight = (self.comp @ weight.view(self.num_bases, -1)).view(
                self.num_relations, self.in_channels_l, self.out_channels)

        if self.num_blocks is not None:  # Block-diagonal-decomposition =======
            if x_j.dtype == torch.long:
                raise ValueError('Block-diagonal decomposition not supported '
                                 'for non-continuous input features.')

            weight = weight[edge_type].view(-1, weight.size(2), weight.size(3))
            x_j = x_j.view(-1, 1, weight.size(1))
            return torch.bmm(x_j, weight).view(-1, self.out_channels)

        else:  # No regularization/Basis-decomposition ========================
            if x_j.dtype == torch.long:
                weight_index = edge_type * weight.size(1) + index
                return weight.view(-1, self.out_channels)[weight_index]

            return torch.bmm(x_j.unsqueeze(-2), weight[edge_type]).squeeze(-2)

    def aggregate(self, inputs: Tensor, edge_type: Tensor, index: Tensor,
                  dim_size: Optional[int] = None) -> Tensor:

        # Compute normalization in separation for each `edge_type`.
        if self.aggr == 'mean':
            norm = F.one_hot(edge_type, self.num_relations).to(torch.float)
            norm = scatter(norm, index, dim=0, dim_size=dim_size)[index]
            norm = torch.gather(norm, 1, edge_type.view(-1, 1))
            norm = 1. / norm.clamp_(1.)
            inputs = norm * inputs

        return scatter(inputs, index, dim=self.node_dim, dim_size=dim_size)


In [70]:
import math
import torch
from torch_geometric.utils import to_undirected


def train_test_split_edges(data, val_ratio=0.05, test_ratio=0.1):
    r"""Splits the edges of a :obj:`torch_geometric.data.Data` object
    into positive and negative train/val/test edges, and adds attributes of
    `train_pos_edge_index`, `train_neg_adj_mask`, `val_pos_edge_index`,
    `val_neg_edge_index`, `test_pos_edge_index`, and `test_neg_edge_index`
    to :attr:`data`.

    Args:
        data (Data): The data object.
        val_ratio (float, optional): The ratio of positive validation
            edges. (default: :obj:`0.05`)
        test_ratio (float, optional): The ratio of positive test
            edges. (default: :obj:`0.1`)

    :rtype: :class:`torch_geometric.data.Data`
    """

    assert 'batch' not in data  # No batch-mode.
    edge_list = data.edge_list
    num_nodes = data.num_nodes
    row, col = data.edge_index
    data.edge_index = None

    # Return upper triangular portion.
    mask = row < col
    row, col = row[mask], col[mask]

    n_v = int(math.floor(val_ratio * row.size(0)))
    n_t = int(math.floor(test_ratio * row.size(0)))

    # Positive edges.
    perm = torch.randperm(row.size(0))
    row, col = row[perm], col[perm]

    r, c = row[:n_v], col[:n_v]
    data.val_pos_edge_index = torch.stack([r, c], dim=0)
    r, c = row[n_v:n_v + n_t], col[n_v:n_v + n_t]
    data.test_pos_edge_index = torch.stack([r, c], dim=0)

    data.train_pos_edge_index_edge_type = []
    r, c = row[n_v + n_t:], col[n_v + n_t:]
    #print()
    data.train_pos_edge_index = torch.stack([r, c], dim=0)
    data.train_pos_edge_index = to_undirected(data.train_pos_edge_index)
  #  print(edge_list[:10])
    print(edge_list[0])
    #print(edge_list[1])
   # print(edge_list[2])
    printtrain_pos_edge_index[0]()
    for sub,rel,obj in zip(edge_list[0], edge_list[1], edge_list[2]):
       index = None
       # print( data.train_pos_edge_index[0].tolist().index(sub))
       if sub in data.train_pos_edge_index[0]:
        index =  data.train_pos_edge_index[0].tolist().index(sub) 
       print("Index is" + str(index))
       #print(data.train_pos_edge_index[1][index])
       #print(data.train_pos_edge_index[0][index])

       if index is not None:
          print("Index is not None")
          if obj == data.train_pos_edge_index[1].tolist()[index]:
            print("obj is " + str(obj) + "sub is: " + str(sub) +  "relation is " + str(rel))
            print("Obj and subj match found")
            print(data.train_pos_edge_index[1][index])
            data.train_pos_edge_index_edge_type.append(rel)
    
    data.train_pos_edge_index_edge_type =  torch.tensor(data.train_pos_edge_index_edge_type, dtype=torch.long).t().contiguous()
    
    # Negative edges.
    neg_adj_mask = torch.ones(num_nodes, num_nodes, dtype=torch.uint8)
    neg_adj_mask = neg_adj_mask.triu(diagonal=1).to(torch.bool)
    neg_adj_mask[row, col] = 0

    neg_row, neg_col = neg_adj_mask.nonzero(as_tuple=False).t()
    perm = torch.randperm(neg_row.size(0))[:n_v + n_t]
    neg_row, neg_col = neg_row[perm], neg_col[perm]

    neg_adj_mask[neg_row, neg_col] = 0
    data.train_neg_adj_mask = neg_adj_mask

    row, col = neg_row[:n_v], neg_col[:n_v]
    data.val_neg_edge_index = torch.stack([row, col], dim=0)

    row, col = neg_row[n_v:n_v + n_t], neg_col[n_v:n_v + n_t]
    data.test_neg_edge_index = torch.stack([row, col], dim=0)

    return data

In [67]:
from torch_geometric.data import (InMemoryDataset, Data)
from collections import Counter
import numpy 
import torch.nn
from sklearn.model_selection import train_test_split
fc = torch.nn.Linear(768, 10)

class EntitiesIOSPress(InMemoryDataset):    


    def __init__(self):
        super(EntitiesIOSPress, self)
        self.data = None

    @property
    def num_relations(self):
        return self.data.edge_type.max().item() + 1

    @property
    def num_features(self):
        return self.data.tmp_num_features


    #def num_classes(self):
        #return self.data.train_y.max().item() + 1

    def process(self):
        g = rdf.Graph()
        graph = g.parse(location='/content/updated_results.ttl', format='ttl')
        freq_ = Counter(graph.predicates())

        def freq(rel):
          return freq_[rel] if rel in freq_ else 0

        relations = sorted(set(g.predicates()), key=lambda rel: -freq(rel)) 
        nodes = {}
        #returns the  authors
        for e1,r,e2 in graph.triples((None, rdf.term.URIRef('https://schema.org/fullName'), None)):
          author_embeddings = torch.from_numpy(bc.encode([e2]))
          nodes[e1] = author_embeddings
        for e1,r,e2 in graph.triples((None, rdf.term.URIRef('https://schema.org/coAuthorOf'), None)):
          if graph.value(subject = e2, predicate = rdf.term.URIRef('https://schema.org/fullName')) is not None:
            author_embeddings = torch.from_numpy(bc.encode([graph.value(subject = e2, predicate = rdf.term.URIRef('https://schema.org/fullName'))]))
            nodes[e2] = author_embeddings
          else:
            graph.remove((e1,r,e2))
        #returns the articles
        for e1,r,e2 in graph.triples((None, rdf.term.URIRef('https://schema.org/hasPubAbstract'), None)):
          article_attribute_embeddings = []
          abstract = e2
          title = graph.value(subject = e1, predicate = rdf.term.URIRef('https://schema.org/hasPubTitle'))
          keywords = graph.value(subject = e1, predicate = rdf.term.URIRef('https://schema.org/hasKeywords'))
          if abstract is not None:
            abstract_embedding = bc.encode([abstract])
            article_attribute_embedding = abstract_embedding
            #print(abstract_embedding.shape)
            #print(abstract_embedding.dtype)
          if title is not None: 
            title_embedding = bc.encode([title])
           # print(title_embedding.shape)
           # print(title_embedding.dtype)
            if len(article_attribute_embedding) > 0:
             # print("1st concat: torch.cat() start for article attribute")
              article_attribute_embedding = torch.cat([torch.from_numpy(article_attribute_embedding), torch.from_numpy(title_embedding)])
             # print("torch.cat() finished for article attribute")
            else : article_attribute_embedding = torch.from_numpy(title_embedding)
          if keywords is not None: 
            #print(keywords)
           # print("Start torch.sum()")
            keywords_embedding = torch.cat([torch.from_numpy(bc.encode([kw])) for kw in keywords if kw is not ' '])
           # print("Finished torch.sum()")
            if len(article_attribute_embedding) > 0:
            #  print("Start 2nd concat")
              article_attribute_embedding = torch.cat([article_attribute_embedding, keywords_embedding])
            #  print("Done torch.cat()")
            else : article_attribute_embedding = torch.from_numpy(keywords_embedding)
         # print("Done with all torch.cat()")
          #pass through dense layer: TODO
          #article_attribute_embedding = torch.cat([bc.encode([abstract]), bc.encode([title]), torch.sum([bc.encode([keywords]) for kw in ])])#pass through dense layer
          nodes[e1] = article_attribute_embedding
        #returns the affiliations
        for e1,r,e2 in graph.triples((None, rdf.term.URIRef('https://schema.org/hasName'), None)):
          affiliation_embedding = []
          affiliation_embedding =  torch.from_numpy(bc.encode([e2]))
          nodes[e1] = affiliation_embedding
        #print(article_attribute_embedding.shape)
        nodes_dict = {key: nodeID for nodeID, key in enumerate(nodes)}
        relations_dict = {rel: i for i, rel in enumerate(list(relations))}
       # print(list(nodes_dict.items())[:5])
       # print(relations_dict)
        embeddings = []
        edge_list = []
        labels = {}
        labels_set = set()
        for e1,r,e2 in graph.triples((None, None, None)):
          if r == rdf.term.URIRef('https://schema.org/hasName'):
              #print((e1,r,e2) + str("Got it"))
              graph.remove((e1,r,e2))
          if r == rdf.term.URIRef('https://schema.org/hasPubTitle'):
              #print((e1,r,e2) + str("Got it"))
              graph.remove((e1,r,e2))
          if r == rdf.term.URIRef('https://schema.org/hasKeywords'):
              #print(str(e1,r,e2) + str("Got it"))
              graph.remove((e1,r,e2))
          if r == rdf.term.URIRef('https://schema.org/fullName'):
              #print((e1,r,e2) + str("Got it"))
              graph.remove((e1,r,e2))
          if r == rdf.term.URIRef('https://schema.org/hasPubAbstract'):
              #print((e1,r,e2) + str("Got it"))
              graph.remove((e1,r,e2)) 
          if r == rdf.term.URIRef('https://schema.org/hasAffiliation'):
              #print((e1,r,e2) + str("Got it"))
              graph.remove((e1,r,e2))
        for e1,r,e2 in graph.triples((None, None, None)):
          if (e1 and r and e2) is not None:
                #print((e1,r,e2))
                src, dst,rel = nodes_dict[e1], nodes_dict[e2], relations_dict[r]
                edge_list.append([src, dst, 2 * rel])
                edge_list.append([dst, src, 2 * rel + 1])
               # print(nodes[e1].size())
                #input_e1 = nodes[e1].view(batch_size, -1)
                #input_e2 = nodes[e2].view(batch_size, -1)
                #print(input_e1)
                output_e1 = fc(nodes[e1])
                output_e2 = fc(nodes[e2])
                #print(output_e1.dtype)
                #print(output_e1.size()) 
                embeddings.append([output_e1 ,output_e2]) #first tuple is the source and the second is dst as tensors.
                embeddings.append([output_e2 ,output_e1])
                #labels[src] = relations_dict[r]
                #labels_set.add(labels[src])
        #
        # Simulate a 28 x 28 pixel, grayscale "image"
       # input = torch.randn(1, 28, 28)
        # Use view() to get [batch_size, num_features].
        # -1 calculates the missing value given the other dim.
       # input = input.view(batch_size, -1) # torch.Size([1, 784])
        # Intialize the linear layer.
       #
        # Pass in the simulated image to the layer.
        #output = fc(embeddings)
       # print(output.shape)
     #   labels_dict = {lab: i for i, lab in enumerate(list(labels_set))}
     #   train_labels, tpytorch geometric
       #  est_labels = list(labels_dict.values())[:80,:], list(labels_dict.values())[80:,:]
       # train_y = torch.tensor(train_labels, dtype=torch.long)
        # IF for loop for edge_list edge_list = sorted(edge_list, key=lambda x: (x[0], x[1], x[2])) # deterministic 
        #TO ADD: for loop for edge_list
        edge = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
        embeddings = torch.tensor(embeddings, dtype=torch.long).t().contiguous()
        edge_index, edge_type = edge[:2], edge[2]
      #  embeddings = output
        #print(embeddings[:3])
        #print(edge_index.shape)
        #print(edge_index[:3])

        data = Data(edge_index=edge_index)
        data.embeddings = embeddings
        data.edge_list = edge     
        data.edge_index = edge_index
        data.edge_type = edge_type 
        #data.train_y = train_y
        data.num_relations = edge_type.max().item() + 1
        data.num_classes = 3
        data.num_nodes = data.edge_index.max().item() + 1
        self.data = data

In [72]:
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score


#logging.basicConfig(filename='Link_pred_simone.log', filemode='w', format='%(asctime)s %(message)s')
dataset = EntitiesIOSPress()
dataset.train_mask = dataset.val_mask = dataset.test_mask = dataset.y = None
dataset.process()
print('done processing')
data = train_test_split_edges(dataset.data)
print('done edges')
print(data)
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = RGCNConv(data.num_nodes, 16, data.num_relations,
                                num_bases=5)
        self.conv2 = RGCNConv(16, data.num_classes, data.num_relations,
                              num_bases=5)

    def encode(self, edge_index, edge_type):
        #print("Edge_index dim: " + str(data.edge_index)) returns None???
        #logging.warning(f'self.conv1(data.x = {data.x}, data.train_pos_edge_index_shape = {data.train_pos_edge_index.shape}, edge_type_shape = {(edge_type.shape)})')
        x = self.conv1(None, edge_index, edge_type)
        #logging.warning(f'self.conv1(data.train_pos_edge_index_shape = {data.edge_index.shape}, edge_type_shape = {(edge_type.shape)}) and x.relu() = {x.relu()}')
        x.relu() 
        return self.conv2(x, edge_index, edge_type)


    def decode(self, z, pos_edge_index, neg_edge_index):
         edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1)
         return (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)

def get_link_labels(pos_edge_index, neg_edge_index):
    E = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(E, dtype=torch.float, device=device)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels


def train(data):
    model.train()

    neg_edge_index = negative_sampling(
        edge_index=data.train_pos_edge_index, num_nodes=data.num_nodes,
        num_neg_samples=data.train_pos_edge_index.size(1))
    optimizer.zero_grad()

    #logging.warning(f'edge_type_shape_in_training = {(data.edge_type.shape)})')
    z = model.encode(data.train_pos_edge_index, data.train_pos_edge_index_edge_type)
    link_logits = model.decode(z, data.train_pos_edge_index, neg_edge_index)
    link_labels = get_link_labels(data.train_pos_edge_index, neg_edge_index)
    loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
    link_probs = link_logits.sigmoid()
    #print("Training roc auc =" + str(roc_auc_score(link_labels.cpu(), link_probs.cpu())))
    loss.backward()
    optimizer.step()
    #print("Loss is:" + str(loss))
    return loss


@torch.no_grad()
def test(data):
    model.eval()

    z = model.encode(data.train_pos_edge_index, data.train_pos_edge_index_edge_type)

    results = []
    for prefix in ["val", "test"]:
        pos_edge_index = data[f'{prefix}_pos_edge_index']
        neg_edge_index = data[f'{prefix}_neg_edge_index']
        link_logits = model.decode(z, pos_edge_index, neg_edge_index)
        #print(str(link_logits) + "this is link_logits")
        link_probs = link_logits.sigmoid()
        link_labels = get_link_labels(pos_edge_index, neg_edge_index)
        results.append(roc_auc_score(link_labels.cpu(), link_probs.cpu()))
        #print("results is: " + str(results))
    return results

best_val_auc = test_auc = 0
for epoch in range(1, 101):
    #print('Start Training')
    train_loss = train(data)
    #print('Finish Training')
    #print("Start Test Phase")
    val_auc, tmp_test_auc = test(data)
    #print("Finish Test Phase")
    #print("val_auc: " + str(val_auc) + "> " + str(best_val_auc) + "best_value_auc")
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        test_auc= tmp_test_auc
    log = 'Epoch: {:03d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    #print("Epoch " + str(epoch))
    print(log.format(epoch, train_loss, best_val_auc, test_auc))

z = model.encode(data.train_pos_edge_index,data.train_pos_edge_index_edge_type)
final_edge_index = model.decode_all(z)


KeyboardInterrupt: ignored